In [ ]:
!nvidia-smi

Mon Sep 13 12:59:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install grpcio-tools==1.34.1
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-ick95sc1
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-ick95sc1


In [ ]:
from typing import List
import pandas as pd
from haystack import Document
from haystack.document_store.faiss import FAISSDocumentStore
from haystack.generator.transformers import RAGenerator
from haystack.retriever.dense import DensePassageRetriever

In [ ]:
!wget -c https://raw.githubusercontent.com/deepset-ai/haystack/master/tutorials/small_generator_dataset.csv

--2021-09-13 13:06:38--  https://raw.githubusercontent.com/deepset-ai/haystack/master/tutorials/small_generator_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48362 (47K) [text/plain]
Saving to: ‘small_generator_dataset.csv’

small_generator_dat 100%[===================>]  47.23K  --.-KB/s    in 0.01s   

2021-09-13 13:06:39 (4.14 MB/s) - ‘small_generator_dataset.csv’ saved [48362/48362]



In [ ]:
df = pd.read_csv("small_generator_dataset.csv", sep=',')
df.fillna(value="", inplace=True)
df.head()

,title,text
0,"""Albert Einstein""",to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ...
1,"""Albert Einstein""",Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a Ge...
2,"""Albert Einstein""","observations were published in the international media, making Einstein worl..."
3,"""Albert Einstein""",model for depictions of mad scientists and absent-minded professors; his exp...
4,"""Alfred Nobel""","was adopted as the standard technology for mining in the ""Age of Engineering..."


In [ ]:
text_sample = df.iloc[0]['text']
text_sample

'to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass–energy equivalence formula , which has been dubbed "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of'

In [ ]:
title_sample = df.iloc[0]['title']
title_sample

'"Albert Einstein"'

In [ ]:
Document(
    text=text_sample,
    meta={"name": title_sample or ""}
)

{'text': 'to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass–energy equivalence formula , which has been dubbed "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of', 'score': None, 'question': None, 'meta': {'name': '"Albert Einstein"'}, 'embedding': None, 'id': 'ab6983f94425886f00fd550b2ac0f037'}

In [ ]:
titles = list(df["title"].values)
texts = list(df["text"].values)
documents: List[Document] = []

for title, text in zip(titles, texts):
    documents.append(
        Document(
            text=text,
            meta={
                "name": title or ""
            }
        )
    )

In [ ]:
documents[:5]

[{'text': 'to Einstein in 1922. Footnotes Citations Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass–energy equivalence formula , which has been dubbed "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of', 'score': None, 'question': None, 'meta': {'name': '"Albert Einstein"'}, 'embedding': None, 'id': 'ab6983f94425886f00fd550b2ac0f037'},
 {'text': 'Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also 

In [ ]:
len(documents)

75

In [ ]:
document_store = FAISSDocumentStore(
    faiss_index_factory_str="Flat",
    return_embedding=True
)

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True,
    embed_title=True,
)

generator = RAGenerator(
    model_name_or_path="facebook/rag-token-nq",
    use_gpu=True,
    top_k=1,
    max_length=200,
    min_length=2,
    embed_title=True,
    num_beams=2,
)

09/13/2021 13:27:07 - INFO - filelock -   Lock 140091222373392 acquired on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

09/13/2021 13:27:08 - INFO - filelock -   Lock 140091222373392 released on /root/.cache/huggingface/transformers/4ad08b5f983c1384baaf257d8edf51a7a3961fd8c75a1778ac604e3c0b564dd9.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
09/13/2021 13:27:09 - INFO - filelock -   Lock 140091220933456 acquired on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

09/13/2021 13:27:11 - INFO - filelock -   Lock 140091220933456 released on /root/.cache/huggingface/transformers/b305bc9085b3d0ce33551c251b75c11b6c6df1d4d51e5d3439d01cf4bb1abc9d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
09/13/2021 13:27:13 - INFO - filelock -   Lock 140091220860688 acquired on /root/.cache/huggingface/transformers/d5b5f07ee846d5baa7142e121b6ee77d11ac68bd5d4541faab38a1ea76c2954a.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


...

09/13/2021 13:28:23 - INFO - filelock -   Lock 140091179555152 released on /root/.cache/huggingface/transformers/73d054acda2f2b44f075a0d391ed392e6c53f78356bbf279581786ae35a79e4d.cb2244924ab24d706b02fd7fcedaea4531566537687a539ebb94db511fd122a0.lock
09/13/2021 13:28:24 - INFO - filelock -   Lock 140091182113872 acquired on /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock


Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

09/13/2021 13:28:24 - INFO - filelock -   Lock 140091182113872 released on /root/.cache/huggingface/transformers/a2f0711fd4c199beaf3ff2626c4a723e107e79a8d065099a3d24e7aa361c0f18.67d01b18f2079bd75eac0b2f2e7235768c7f26bd728e7a855a1c5acae01a91a8.lock
09/13/2021 13:28:26 - INFO - filelock -   Lock 140091179648208 acquired on /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6.lock


Downloading:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

09/13/2021 13:31:42 - INFO - filelock -   Lock 140091179648208 released on /root/.cache/huggingface/transformers/6288b70ae87b5989f6801120b25a2f6ccf2e88c2f622d94805717b5b043ff71b.bceb1ea6e95a1b39d0a717f315800a39b0847e5cdeeb42615fb498775f632ee6.lock
Some weights of RagTokenForGeneration were not initialized from the model checkpoint at facebook/rag-token-nq and are newly initialized: ['rag.generator.lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
document_store.delete_documents()

document_store.write_documents(documents)

document_store.update_embeddings(
    retriever=retriever
)

09/13/2021 13:32:16 - INFO - haystack.document_store.faiss -   Updating embeddings for 68 docs...
Updating Embedding:   0%|          | 0/68 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/80 [00:00<?, ? Docs/s]

Documents Processed: 10000 docs [00:02, 3800.29 docs/s]


In [ ]:
question = "who got the first nobel prize in physics"

retriever_results = retriever.retrieve(
    query=question
)

In [ ]:
retriever_results[0]

{'text': 'Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass–energy equivalence formula , which has been dubbed "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step', 'score': 0.6908883762847904, 'question': None, 'meta': {'name': '"Albert Einstein"', 'vector_id': '1'}, 'embedding': array([ 8.55912268e-02,  2.62766957e-01, -1.33803874e-01,  2.22948082e-02,
       -2.07061455e-01,  3.82364482e-01,  7.28562921e-02, -2.72492379e-01,
       -1.08158864e-01, -2.91012913e-01, -4.35574621e-01, -2.45857641e-01,
       -5.21386981e-01,  9.44196999e-01,

In [ ]:
predicted_result = generator.predict(
    query=question,
    documents=retriever_results,
    top_k=1
)

/usr/local/lib/python3.7/dist-packages/transformers/models/rag/tokenization_rag.py:97: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1714: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,


In [ ]:
predicted_result

{'answers': [{'answer': ' albert einstein',
   'meta': {'doc_ids': ['b85487f3f7fd1b9d5d4f4ed0aea4ef09',
     'ab6983f94425886f00fd550b2ac0f037',
     'd34ecd3aeacc8cbd33555e07f9a98c73',
     '4a347e0fc7c1b7d06fa7d7a2aa580555',
     '549e4d922dd841db8244599f1d9ac30c',
     '3c527839680f7f5ff3cb758454fa5079',
     '2aedb0c40e3a20c805d5b404a39543a',
     '82fe278b9f77e6e24998b5c17dc3dbf8',
     '66324728ba390547d45ac16765ce6fb5',
     '40ed0bdd0f9da67b94830eab87593552'],
    'doc_scores': [0.6908883762847904,
     0.6899226137320913,
     0.6845092575712043,
     0.683915370112666,
     0.6816012020544886,
     0.6551759945377117,
     0.6422037785182146,
     0.6376394847804104,
     0.6324816414259261,
     0.6317146301754861],
    'texts': ['Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for i

In [ ]:
answers = predicted_result["answers"]
answers

[{'answer': ' albert einstein',
  'meta': {'doc_ids': ['b85487f3f7fd1b9d5d4f4ed0aea4ef09',
    'ab6983f94425886f00fd550b2ac0f037',
    'd34ecd3aeacc8cbd33555e07f9a98c73',
    '4a347e0fc7c1b7d06fa7d7a2aa580555',
    '549e4d922dd841db8244599f1d9ac30c',
    '3c527839680f7f5ff3cb758454fa5079',
    '2aedb0c40e3a20c805d5b404a39543a',
    '82fe278b9f77e6e24998b5c17dc3dbf8',
    '66324728ba390547d45ac16765ce6fb5',
    '40ed0bdd0f9da67b94830eab87593552'],
   'doc_scores': [0.6908883762847904,
    0.6899226137320913,
    0.6845092575712043,
    0.683915370112666,
    0.6816012020544886,
    0.6551759945377117,
    0.6422037785182146,
    0.6376394847804104,
    0.6324816414259261,
    0.6317146301754861],
   'texts': ['Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of

In [ ]:
question = "what is the most current adobe flash player version"

retriever_results = retriever.retrieve(
    query=question
)

predicted_result = generator.predict(
    query=question,
    documents=retriever_results,
    top_k=1
)

answers = predicted_result["answers"]
answers

/usr/local/lib/python3.7/dist-packages/transformers/models/rag/tokenization_rag.py:97: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1714: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,


[{'answer': ' 7.1. 2',
  'meta': {'doc_ids': ['ce27ab4c90c16d47bf3577fac072ce12',
    '792312bbd1b1e8c8c4365b5966dee573',
    'c38368e3e17050542f6f999d57296ffb',
    '9dfc548a82f643420f5a3fa219088773',
    '69c8b13a944a0878822e417f028aae49',
    'bd3e02e3146be9bc4081cc806bbee7af',
    '23c07d7b51ea44a1db9af1265ac6c276',
    'e86ca2c67ea5220c1bad5de0868fad3f',
    '4bc744a58c35759e701f569c3afbec12',
    '67916365e3f6bbfc35a1a234233dc0dc'],
   'doc_scores': [0.6708526497104501,
    0.6663908371332308,
    0.6639012445000928,
    0.6637408088753275,
    0.6610911881480432,
    0.6596591712821107,
    0.6567510128617647,
    0.6525359875281581,
    0.6479908012327465,
    0.6439680482885661],
   'texts': ['include access to the iTunes Store to rent movies and TV shows (purchasing has been discontinued), streaming from internet video sources, including YouTube and Netflix, and media streaming from an iTunes library. Apple also reduced the price of the device to $99. A third generation of th

In [ ]:
from haystack.pipeline import GenerativeQAPipeline

In [ ]:
pipe = GenerativeQAPipeline(generator=generator, retriever=retriever)
res = pipe.run(query=question, params={"Generator": {"top_k": 1}, "Retriever": {"top_k": 5}})
print(res)

/usr/local/lib/python3.7/dist-packages/transformers/models/rag/tokenization_rag.py:97: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1714: UserWarning: `max_length` is deprecated in this function, use `stopping_criteria=StoppingCriteriaList(MaxLengthCriteria(max_length=max_length))` instead.
  UserWarning,


{'query': 'who got the first nobel prize in physics', 'answers': [{'query': 'who got the first nobel prize in physics', 'answer': ' albert einstein', 'meta': {'doc_ids': ['b85487f3f7fd1b9d5d4f4ed0aea4ef09', 'ab6983f94425886f00fd550b2ac0f037', 'd34ecd3aeacc8cbd33555e07f9a98c73', '4a347e0fc7c1b7d06fa7d7a2aa580555', '549e4d922dd841db8244599f1d9ac30c'], 'doc_scores': [0.6908883762847904, 0.6899226137320913, 0.6845092575712043, 0.683915370112666, 0.6816012020544886], 'texts': ['Albert Einstein Albert Einstein (; ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who developed the theory of relativity, one of the two pillars of modern physics (alongside quantum mechanics). His work is also known for its influence on the philosophy of science. He is best known to the general public for his mass–energy equivalence formula , which has been dubbed "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and e